## About the Game

The rules of Skid Isolation are a variation of the original Isolation. In the original form of the game there are two players, each with their own game piece, and a 7-by-7 grid of squares. At the beginning of the game, the first player places their piece on any square. The second player follows suit, and places their piece on any one of the available squares. From that point on, the players alternate turns moving their piece like a queen in chess (any number of open squares vertically, horizontally, or diagonally). When the piece is moved, the square that was previously occupied is blocked, and cannot be used for the remainder of the game. The first player who is unable to move their queen loses.

In this variant called the Skid Isolation, each player leaves behind traces when they move their queen. We say the piece/queen "skids", and as a result the skid places a permanent block in the previously occupied block and the block "just before" the player's current position and the block "just after" the player's previous position. The opponent cannot move on or through squares blocked by this skid. For clarity, examine the scenario below:

Q1 places their queen on the board, and Q2 follows suit.

<div>
<img src="./img/1.png" width="500"/>
</div>

Q1 makes a diagonal move across the board and leaves behind a skid, blocking some of Q2's potential future moves.

<div>
<img src="./img/2.png" width="500"/>
</div>

Q2 makes a move, leaving behind her own skid, blocking some of Q1's future potential moves, as well.

<div>
<img src="./img/3.png" width="500"/>
</div>

The game will continue in a similar fashion until either of the queens has no cells left to move.



In [4]:
%run helpers/verify_config.py # verify the environment setup

Your python version is  3.7.13
⚠️ Library version conflict
(pgmpy 0.1.10 (c:\users\83509\anaconda3\envs\ai_env\lib\site-packages), Requirement.parse('pgmpy==0.1.9'))


In [5]:
%load_ext autoreload
%autoreload 2
from board_viz import ReplayGame, InteractiveGame
from isolation import Board
from test_players import RandomPlayer

In [6]:
ig = InteractiveGame(None, show_legal_moves=True)

GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001', height='auto', width='au…

# Important Files

1. `isolation.py`: Includes the `Board` class and a function for printing out a game as text. 
2. `notebook.ipynb`: Game agents.
3. `player_submission_tests.py`: Sample tests to validate game agents locally.
4. `test_players.py`: Contains 2 player types to test agents locally:
    - `RandomPlayer` - chooses a legal move randomly from among the available legal moves
    - `HumanPlayer` - play against the AI in terminal (else use `InteractiveGame` in jupyter)

### Evaluation Functions

These functions will inform the value judgements your AI will make when choosing moves. There are 2 classes:

- `OpenMoveEvalFn` -Returns the number of available moves open for your player minus the number of moves available for opponent player. All baseline tests will use this function.
- `CustomEvalFn` - Customized evaluation function

## Importing External Modules

In [9]:
%load_ext autoreload
%autoreload 2
import player_submission_tests as tests
from test_players import HumanPlayer, RandomPlayer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import time
from isolation import Board

In [14]:
class OpenMoveEvalFn:
    def score(self, game, my_player=None):
        """Score the current game state
        Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

        Note:
            If you think of better evaluation function, do it in CustomEvalFn below.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """

#         if len(game.get_player_moves(my_player)) == 0:
#                 return float(-1e9)
            
#         if len(game.get_opponent_moves(my_player)) == 0:
#             return float(1e9)
        
        player_moves = len(game.get_player_moves(my_player))
        opponent_moves = len(game.get_opponent_moves(my_player))
        
        return float(player_moves-opponent_moves)
        
tests.correctOpenEvalFn(OpenMoveEvalFn)


OpenMoveEvalFn Test: This board has a score of -9.0.



In [30]:
class CustomPlayer:

    def __init__(self, search_depth=3, eval_fn=CustomEvalFn()):
        """
        Args:
            search_depth (int): The depth to which your agent will search
            eval_fn (function): Evaluation function used by your agent
        """
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        best_move, utility = minimax(self, game, time_left, depth=self.search_depth)
#         best_move, utility = alphabeta(self, game, time_left, depth=self.search_depth, alpha=float("-inf"), beta=float("inf"), my_turn=True)
        return best_move

    def utility(self, game, my_turn):
        
        score = self.eval_fn.score(game, self)
        return score
        if my_turn:
            return score
        else:
            return -score


## Minimax

In [27]:
def minimax(player, game, time_left, depth, my_turn=True):
    """
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents game agent. 
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep agent is in the search tree
        my_turn (bool): True if agent is computing scores during human player's turn.

    Returns:
        (tuple, int): best_move, val
    """

    if my_turn:
        moves = game.get_active_moves()
        best_move = moves[0]
        best_score = float('-inf')
        for move in moves:
            board, ff, winner = game.forecast_move(move)
            score = minimax_minvalue(player, board, ff, winner, depth-1, time_left, not my_turn)
            if score > best_score:
                best_move = move
                best_score = score
        return best_move, int(best_score)
    else:
        moves = game.get_active_moves()
        best_move = moves[0]
        best_score = float('inf')
        for move in moves:
            board, ff, winner = game.forecast_move(move)
            score = minimax_maxvalue(player, board, ff, winner, depth-1, time_left, not my_turn)
            if score < best_score:
                best_move = move
                best_score = score
        return best_move, int(best_score)


def minimax_maxvalue(player, board, ff, winner, depth, time_left, my_turn):
    if depth == 0 or time_left() < 500 or ff:
        return player.utility(board, my_turn)

    moves = board.get_active_moves()
    best_score = float('-inf')
    for move in moves:
        nboard, nff, nwinner = board.forecast_move(move)
        score = minimax_minvalue(player, nboard, nff, nwinner, depth - 1, time_left, not my_turn)
        if score > best_score:
            best_score = score
    return best_score


def minimax_minvalue(player, board, ff, winner, depth, time_left, my_turn):
    if depth == 0 or time_left() < 500 or ff:
        return player.utility(board, my_turn)

    moves = board.get_active_moves()
    best_score = float('inf')
    for move in moves:
        nboard, nff, nwinner = board.forecast_move(move)
        score = minimax_maxvalue(player, nboard, nff, nwinner, depth - 1, time_left, not my_turn)
        if score < best_score:
            best_score = score
    return best_score

tests.beatRandom(CustomPlayer)
tests.minimaxTest(CustomPlayer, minimax)  




 RandomPlayer - Q1  Turn
move chosen:  (5, 6)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |Q1|
6 |  |  |  |  |  |  |  |

 CustomPlayer - Q2  Turn
move chosen:  (6, 0)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |Q1|
6 |Q2|  |  |  |  |  |  |

 RandomPlayer - Q1  Turn
move chosen:  (2, 3)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |  |  |Q1|  |  |  |
3 |  |  |  |  |><|  |  |
4 |  |  |  |  |  |><|  |
5 |  |  |  |  |  |  |><|
6 |Q2|  |  |  |  |  |  |

 CustomPlayer - Q2  Turn
move chosen:  (2, 0)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |Q2|  |  |Q1|  |  |  |
3 |><|  |  |  |><|  |  |
4 |  |  |  |  |  |><|  |
5 |><|  |  |  |  |  |><|
6 |><|  |  |  |  |  |  |

 RandomPla

## AlphaBeta Pruning

In [44]:
def alphabeta(player, game, time_left, depth, alpha=float("-inf"), beta=float("inf"), my_turn=True):
    """
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer() 
            that represents game agent.
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep the game agent is in the search tree
        alpha (float): Alpha value for pruning
        beta (float): Beta value for pruning
        my_turn (bool): True if game agent is computing scores during human player's turn.
        
    Returns:
        (tuple, int): best_move, val
    """
    
    if my_turn:
        moves = game.get_active_moves()
        best_move = moves[0]
        for move in moves:
            board, ff, winner = game.forecast_move(move)
            score = ab_minvalue(player, board, ff, winner, depth-1, time_left, alpha, beta, not my_turn)
            if score > alpha:
                alpha = score
                best_move = move
        return best_move, int(score)
    else:
        moves = game.get_active_moves()
        best_move = moves[0]
        for move in moves:
            board, ff, winner = game.forecast_move(move)
            score = ab_maxvalue(player, board, ff, winner, depth-1, timeleft, alpha, beta, not my_turn)
            if score < beta:
                beta = score
                best_move = move
        return best_move, int(score)
    
    
def ab_maxvalue(player, board, ff, winner, depth, time_left, alpha, beta, my_turn):
    if depth == 0 or time_left() < 500 or ff:
        return player.utility(board, my_turn)

    moves = board.get_active_moves()
    for move in moves:
        nboard, nff, nwinner = board.forecast_move(move)
        score = ab_minvalue(player, nboard, nff, nwinner, depth - 1, time_left, alpha, beta, not my_turn)
        if score > alpha:
            alpha = score 
            if alpha >= beta:
                break       
    return alpha


def ab_minvalue(player, board, ff, winner, depth, time_left, alpha, beta, my_turn):
    if depth == 0 or time_left() < 500 or ff:
        return player.utility(board, my_turn)

    moves = board.get_active_moves()
    for move in moves:
        nboard, nff, nwinner = board.forecast_move(move)
        score = ab_maxvalue(player, nboard, nff, nwinner, depth - 1, time_left, alpha, beta, not my_turn)
        if score < beta:
            beta = score 
            if beta <= alpha:
                break       
    return beta

tests.beatRandom(CustomPlayer)




 RandomPlayer - Q1  Turn
move chosen:  (2, 1)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |Q1|  |  |  |  |  |
3 |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |

 CustomPlayer - Q2  Turn
move chosen:  (3, 2)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |Q1|  |  |  |  |  |
3 |  |  |Q2|  |  |  |  |
4 |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |

 RandomPlayer - Q1  Turn
move chosen:  (2, 6)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |><|><|  |  |><|Q1|
3 |  |  |Q2|  |  |  |  |
4 |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |

 CustomPlayer - Q2  Turn
move chosen:  (3, 5)
  |0 |1 |2 |3 |4 |5 |6 |
0 |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |
2 |  |><|><|  |  |><|Q1|
3 |  |  |><|><|><|Q2|  |
4 |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |

 RandomPla

## CustomEvalFn

In [34]:
class CustomEvalFn:
    def __init__(self):
        pass

    def score(self, game, my_player=None):
        """Score the current game state.
        
        Args:
            game (Board): The board and game state.
            my_player (Player object): This specifies which player you are.
            
        Returns:
            float: The current state's score. If in bad situation, choose defensive strategy; 
            if in winning position, choose offensive strategy
        """

        player_moves = len(game.get_player_moves(my_player))
        opponent_moves = len(game.get_opponent_moves(my_player))
        board_size = game.width * game.height
        m = player_moves / board_size
        
        if m > 0.5:
            return float(player_moves-2*opponent_moves)
        else:
            return float(2*player_moves-opponent_moves)
